In [1]:
from multiprocessing import Pool
from functools import partial
import numpy as np
#from numba import jit

In [2]:
#TODO: loss of least square regression and binary logistic regression
'''
    pred() takes GBDT/RF outputs, i.e., the "score", as its inputs, and returns predictions.
    g() is the gradient/1st order derivative, which takes true values "true" and scores as input, and returns gradient.
    h() is the heassian/2nd order derivative, which takes true values "true" and scores as input, and returns hessian.
'''
class leastsquare(object):
    '''Loss class for mse. As for mse, pred function is pred=score.'''
    def pred(self,score):
        return score

    def g(self,true,score):
        return score - true

    def h(self,true,score):
        return np.ones_like(score)

class logistic(object):
    '''Loss class for log loss. As for log loss, pred function is logistic transformation.'''
    def pred(self,score):
        return 1 / (1 + np.exp(-score))

    def g(self,true,score):
        p = self.pred(score)
        return p - true

    def h(self,true,score):
        p = self.pred(score)
        return p * (1 - p)

In [13]:
# TODO: class of Random Forest
class RF(object):
    '''
    Class of Random Forest

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth d_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'mse',
        max_depth = 3, min_sample_split = 10,
        lamda = 1, gamma = 0,
        rf = 0.99, num_trees = 100):

        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.num_trees = num_trees

    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        #TODO
        if self.loss == 'mse':
            self.loss = leastsquare()
        else:
            self.loss = logistic()
        self.estimators = []
        n = train.shape[0]
        for _ in range(self.num_trees):
            estimator = DecisionTree(max_depth = self.max_depth,
                    min_sample_split = self.min_sample_split, gamma = self.gamma)
            indices = np.random.choice(n, n, replace=True)
            train_subset, target_subset = train[indices], target[indices]
            estimator.fit(train_subset, target_subset)
            self.estimators.append(estimator)
        return self

    def predict(self, X):
        predictions = np.array([estimator.predict(X) for estimator in self.estimators])
        if self.loss == 'mse':
            score = np.mean(predictions, axis=0)
        else:
            score = []
            for i in range(predictions.shape[1]):
                sample_predictions = predictions[:, i]
                binary_predictions = (sample_predictions >= 0.5).astype(int)
                max_vote_result = np.bincount(binary_predictions).argmax()
                score.append(max_vote_result)
        return score

In [54]:
# TODO: class of GBDT
class GBDT(object):
    '''
    Class of gradient boosting decision tree (GBDT)

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth D_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        learning_rate: The learning rate eta of GBDT.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'mse',
        max_depth = 3, min_sample_split = 10,
        lamda = 1, gamma = 0,
        learning_rate = 0.1, num_trees = 100):

        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.num_trees = num_trees

    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        #TODO
        self.estimators=[]
        if self.loss=='mse':
            self.loss=leastsquare()
        if self.loss=='log':
            self.loss=logistic()
        self.score_start=target.mean()
        score=np.ones(len(train))*self.score_start
        for i in range(self.num_trees):
            estimator=Tree(n_threads=self.n_threads,
                           max_depth=self.max_depth,min_sample_split=self.min_sample_split,lamda=self.lamda,gamma=self.gamma)
            estimator.fit(train,g=self.loss.g(target,score),h=self.loss.h(target,score))
            self.estimators.append(estimator)
            score+=self.learning_rate*estimator.predict(train)
        return self

    def predict(self, test):
        #TODO
        score=np.ones(len(test))*self.score_start
        for i in range(self.num_trees):
            score+=self.learning_rate*self.estimators[i].predict(test)
        return self.loss.pred(score)

In [68]:
# TODO: class of a node on a tree
class TreeNode(object):
    '''
    Data structure that are used for storing a node on a tree.

    A tree is presented by a set of nested TreeNodes,
    with one TreeNode pointing two child TreeNodes,
    until a tree leaf is reached.

    A node on a tree can be either a leaf node or a non-leaf node.
    '''

    #TODO
    def __init__(self, is_leaf = False, score = None,
                 split_feature = None, split_threshold = None,
                 left_child = None, right_child = None, max_depth = 0):
        self.is_leaf = is_leaf
        self.score = score
        self.split_feature = split_feature
        self.split_threshold = split_threshold
        self.left_child = left_child
        self.right_child = right_child

In [86]:
# TODO: class of single tree
class Tree(object):
    '''
    Class of a single decision tree in GBDT

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        max_depth: The maximum depth of the tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf prediction, also known as lambda.
        gamma: The regularization coefficient for number of TreeNode, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule,
            rf = 0 means we are training a GBDT.
    '''

    def __init__(self, n_threads = None,
                 max_depth = 3, min_sample_split = 10,
                 lamda = 1, gamma = 0, rf = 0):
        self.n_threads = n_threads
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.root = None

    def fit(self, train, g, h):
        '''
        train is the training data matrix, and must be numpy array (an n_train x m matrix).
        g and h are gradient and hessian respectively.
        '''
        #TODO
        self.estimator = self.construct_tree(train, g, h, self.max_depth)
        return self

    def predict(self,test):
        '''
        test is the test data matrix, and must be numpy arrays (an n_test x m matrix).
        Return predictions (scores) as an array.
        '''
        #TODO
        pool = Pool(self.n_threads)
        f = partial(self.predict_single, self.estimator)
        result = np.array(pool.map(f,test))
        pool.close()
        pool.join()
        return result

    def predict_single(self, treenode, test):
        if treenode.is_leaf:
            return treenode.score
        else:
            if test[treenode.split_feature] < treenode.split_threshold:
                return self.predict_single(treenode.left_child, test)
            else:
                return self.predict_single(treenode.right_child, test)

    def construct_tree(self, train, g, h, max_depth):
        '''
        Tree construction, which is recursively used to grow a tree.
        First we should check if we should stop further splitting.

        The stopping conditions include:
            1. tree reaches max_depth $d_{max}$
            2. The number of sample points at current node is less than min_sample_split, i.e., $n_{min}$
            3. gain <= 0
        '''
        #TODO
        if max_depth == 0 or len(train) < self.min_sample_split:
            return TreeNode(is_leaf=True, score=self.leaf_score(g, h))

        feature, threshold, gain = self.find_best_decision_rule(train, g, h)

        if gain <= self.gamma:
            return TreeNode(is_leaf=True, score=self.leaf_score(g, h))

        index = train[:, feature] < threshold
        left_child = self.construct_tree(train[index], g[index], h[index], max_depth - 1)
        right_child = self.construct_tree(train[~index], g[~index], h[~index], max_depth - 1)
        return TreeNode(split_feature = feature, split_threshold = threshold,
                        left_child = left_child, right_child = right_child)

    def leaf_score(self,g,h):
        return -np.sum(g) / (np.sum(h) + self.lamda)

    def leaf_loss(self, g, h):
        return -0.5 * np.square(np.sum(g)) / (np.sum(h) + self.lamda)

    def find_best_decision_rule(self, train, g, h):
        '''
        Return the best decision rule [feature, treshold], i.e., $(p_j, \tau_j)$ on a node j,
        train is the training data assigned to node j
        g and h are the corresponding 1st and 2nd derivatives for each data point in train
        g and h should be vectors of the same length as the number of data points in train

        for each feature, we find the best threshold by find_threshold(),
        a [threshold, best_gain] list is returned for each feature.
        Then we select the feature with the largest best_gain,
        and return the best decision rule [feature, treshold] together with its gain.
        '''
        #TODO
        pool = Pool(self.n_threads)
        f = partial(self.find_threshold, g, h)
        thresholds=np.array(pool.map(f, train.T))
        pool.close()
        pool.join()
        feature = np.argmax(thresholds[:,1], axis = 0)
        threshold = thresholds[feature, 0]
        gain = thresholds[feature, 1]
        return feature, threshold, gain

    def find_threshold(self, g, h, train):
        '''
        Given a particular feature $p_j$,
        return the best split threshold $\tau_j$ together with the gain that is achieved.
        '''
        #TODO
        loss = self.leaf_loss(g, h)
        threshold = None
        best_gain = 0
        unq = np.unique(train)
        for i in range(1, len(unq)):
            this_threshold = (unq[i - 1] + unq[i]) / 2
            index = train < this_threshold
            left_loss = self.leaf_loss(g[index], h[index])
            right_loss = self.leaf_loss(g[~index], h[~index])
            this_gain = loss - left_loss - right_loss
            if this_gain > best_gain:
                threshold = this_threshold
                best_gain = this_gain
        return [threshold, best_gain]

In [4]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor '''

        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red

        # for leaf node
        self.value = value

In [5]:
class DecisionTree():
    def __init__(self, min_sample_split=2, max_depth=2, gamma=0):
        ''' constructor '''

        # initialize the root of the tree
        self.root = None

        # stopping conditions
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.gamma = gamma
        self.rf = 0.99

    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''

        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_sample_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["var_red"])

        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''

        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        num_features_to_select = int(self.rf * num_features)
        min_rf_m = int(0.2 * num_features_to_select)
        max_rf_m = int(0.5 * num_features_to_select)
        self.rf_m = np.random.randint(min_rf_m, max_rf_m + 1)
        selected_feature_indices = np.random.choice(num_features, self.rf_m, replace=False)
        for feature_index in selected_feature_indices:
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
        # return best split
        return best_split

    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''

        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right

    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''

        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction

    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''

        val = np.mean(Y)
        return val

    def fit(self, X, Y):
        ''' function to train the tree '''

        dataset = np.concatenate((X, Y.reshape(-1, 1)), axis=1)
        self.root = self.build_tree(dataset)

    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''

        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

    def predict(self, X):
        ''' function to predict a single data point '''

        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

In [6]:
# TODO: Evaluation functions (you can use code from previous homeworks)

# RMSE
def root_mean_square_error(pred, y):
    #TODO
    squared_errors = (pred - y) ** 2
    mean_squared_error = np.mean(squared_errors)
    rmse = np.sqrt(mean_squared_error)
    return rmse

# precision
def accuracy(pred, y):
    #TODO
    correct_predictions = np.sum(pred == y)
    total_predictions = len(y)
    acc = correct_predictions / total_predictions
    return acc

In [54]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'max_depth': [2, 3, 5, 7, 10],
#     'min_sample_split': [1, 5, 10, 20, 25, 30, 40, 50],
#     'lamda': [0, 1, 3, 5, 7, 10],
#     'gamma': [0, 0.1, 0.3, 0.5, 0.7, 1],
#     'num_trees': [5, 10, 20, 25, 30, 40, 50],
# }

In [ ]:
# TODO: GBDT regression on boston house price dataset
# Case 1
# load data
import numpy as np
import pandas as pd

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X_1 = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y_1 = raw_df.values[1::2, 2]

# train-test split
from sklearn.model_selection import train_test_split
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.3, random_state=8)
print(X_1.shape, y_1.shape, X_train_1.shape, y_train_1.shape, X_test_1.shape, y_test_1.shape)

In [48]:
# 2.1 RF regression on boston house price dataset

# create a RF regressor
rf_regressor = RF(
    max_depth=3,
    min_sample_split=10,
    gamma=0)

# fit the RF regressor
rf_regressor.fit(X_train_1, y_train_1)

# predict on the training and testing data
y_train_pred_rf = rf_regressor.predict(X_train_1)
y_test_pred_rf = rf_regressor.predict(X_test_1)
train_rmse_rf = root_mean_square_error(y_train_pred_rf, y_train_1)
test_rmse_rf = root_mean_square_error(y_test_pred_rf, y_test_1)

print(f"Training RMSE: {train_rmse_rf}")
print(f"Test RMSE: {test_rmse_rf}")

Training RMSE: 2.5431938298646632
Test RMSE: 3.9066052988975035


In [38]:
# 3.1 GBDT regression on boston house price dataset

# create a GBDT regressor
gbdt_regressor = GBDT(
    max_depth=3,
    min_sample_split=10,
    lamda=1,
    gamma=0,
    learning_rate=0.1,
    num_trees=100
)

# fit the GBDT regressor
gbdt_regressor.fit(X_train_1, y_train_1)

# predict on the training and testing data
y_train_pred_gb_1 = gbdt_regressor.predict(X_train_1)
y_test_pred_gb_1 = gbdt_regressor.predict(X_test_1)
train_rmse_gb_1 = root_mean_square_error(y_train_pred_gb_1, y_train_1)
test_rmse_gb_1 = root_mean_square_error(y_test_pred_gb_1, y_test_1)

print("Training RMSE:", train_rmse_gb_1)
print("Test RMSE:", test_rmse_gb_1)

Training RMSE: 1.3697603513671248
Test RMSE: 3.4842861345840723


In [ ]:
# TODO: GBDT classification on credit-g dataset
# Case 2
# load data
from sklearn.datasets import fetch_openml
data = fetch_openml('credit-g', version=1, return_X_y=True, data_home='credit/')
X_2, y_2 = data[0], data[1]
y_2 = np.array(list(map(lambda x: 1 if x == 'good' else 0, y_2)))

categorical_columns = []
for column in X_2.columns:
    if X_2[column].dtype == 'category':
        categorical_columns.append(column)
X_encoded = np.zeros((X_2.shape[0], 0))
for column in categorical_columns:
    unique_values = np.unique(X_2[column])
    num_unique_values = len(unique_values)
    encoded_columns = np.zeros((X_2.shape[0], num_unique_values))
    for i, value in enumerate(X_2[column]):
        encoded_columns[i, np.where(unique_values == value)] = 1
    X_encoded = np.concatenate((X_encoded, encoded_columns), axis=1)

# train-test split
from sklearn.model_selection import train_test_split
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_encoded, y_2, test_size=0.3, random_state=8)
print(X_encoded.shape, y_2.shape, X_train_2.shape, y_train_2.shape, X_test_2.shape, y_test_2.shape)

In [21]:
# 2.2 RF classification on credit-g dataset

# create a RF classifier
rf_classifier_c = RF(
    loss='log',
    max_depth=10,
    min_sample_split=2,
    gamma=0)

# fit the RF classifier on the training data
rf_classifier_c.fit(X_train_2, y_train_2)

# predict on the training and testing data
y_train_pred_c = rf_classifier_c.predict(X_train_2)
y_test_pred_c = rf_classifier_c.predict(X_test_2)
y_train_pred_c = (y_train_pred_c > 0.5).astype(int)
y_test_pred_c = (y_test_pred_c > 0.5).astype(int)

train_accuracy_c = accuracy(y_train_pred_c, y_train_2)
test_accuracy_c = accuracy(y_test_pred_c, y_test_2)

print(f"Training Accuracy: {train_accuracy_c:.2f}")
print(f"Testing Accuracy: {test_accuracy_c:.2f}")

Training Accuracy:  0.7414285714285714
Test Accuracy: 0.7166666666666667


In [15]:
# 3.2 GBDT classification on credit-g dataset

# create a GBDT classifier
gbdt_classifier = GBDT(
    max_depth=3,
    loss = 'log',
    min_sample_split=10,
    lamda=1,
    gamma=0,
    learning_rate=0.1,
    num_trees=100
)

# fit the GBDT classifier on the training data
gbdt_classifier.fit(X_train_2, y_train_2)

# predict on the training and testing data
y_train_pred = gbdt_classifier.predict(X_train_2)
y_test_pred = gbdt_classifier.predict(X_test_2)
y_train_pred = (y_train_pred > 0.5).astype(int)
y_test_pred = (y_test_pred > 0.5).astype(int)

train_accuracy = accuracy(y_train_pred, y_train_2)
test_accuracy = accuracy(y_test_pred, y_test_2)

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Training Accuracy: 0.8514285714285714
Test Accuracy: 0.7733333333333333


In [ ]:
# TODO: GBDT classification on breast cancer dataset
# Case 3
# load data
from sklearn import datasets
breast_cancer = datasets.load_breast_cancer()
X_3 = breast_cancer.data
y_3 = breast_cancer.target

# train-test split
from sklearn.model_selection import train_test_split
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3, test_size=0.3, random_state=8)
print(X_3.shape, y_3.shape, X_train_3.shape, y_train_3.shape, X_test_3.shape, y_test_3.shape)

In [20]:
# 2.3 RF classification on breast cancer dataset

# create a RF classifier
rf_classifier_b = RF(
    loss='log',
    max_depth=10,
    min_sample_split=2)

# fit the RF classifier on the training data
rf_classifier_b.fit(X_train_3, y_train_3)

# predict on the training and testing data
y_train_pred_b = rf_classifier_b.predict(X_train_3)
y_test_pred_b = rf_classifier_b.predict(X_test_3)
y_train_pred_b = (y_train_pred_b > 0.5).astype(int)
y_test_pred_b = (y_test_pred_b > 0.5).astype(int)

train_accuracy_b = accuracy(y_train_pred_b, y_train_3)
test_accuracy_b = accuracy(y_test_pred_b, y_test_3)

print(f"Training Accuracy: {train_accuracy_b:.2f}")
print(f"Testing Accuracy: {test_accuracy_b:.2f}")

Training Accuracy: 0.9673366834170855
Test Accuracy: 0.9239766081871345


In [20]:
# 3.3 GBDT classification on breast cancer dataset

# create a GBDT classifier
gbdt_classifier = GBDT(
    max_depth=3,
    loss = 'log',
    min_sample_split=10,
    lamda=1,
    gamma=0,
    learning_rate=0.1,
    num_trees=100
)

# fit the GBDT classifier on the training data
gbdt_classifier.fit(X_train_3, y_train_3)

# predict on the training and testing data
y_train_pred = gbdt_classifier.predict(X_train_3)
y_test_pred = gbdt_classifier.predict(X_test_3)
y_train_pred = (y_train_pred > 0.5).astype(int)
y_test_pred = (y_test_pred > 0.5).astype(int)

train_accuracy = accuracy(y_train_pred, y_train_3)
test_accuracy = accuracy(y_test_pred, y_test_3)

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Training Accuracy: 1.0
Test Accuracy: 0.9649122807017544
